### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 5, 23)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-05-23'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,DOHOME,20.30,0,"2,400","7,200",20.25,2022-05-18,2022-05-23,3,19.60,19.00,-6,19.30,25.62,23.29,"48,720.00",I
5,IP,20.30,0,"2,500","7,500",20.20,2022-05-20,2022-05-23,1,20.50,20.40,-1,20.70,24.17,25.25,"50,750.00",I
1,JASIF,12.00,0,"10,000","110,000",10.10,2022-05-17,2022-05-23,4,11.10,10.80,-3,10.90,11.30,11.87,"120,000.00",I
2,MAKRO,40.50,0,"1,500","7,500",40.40,2022-05-19,2022-05-23,2,35.25,36.75,6,36.75,57.25,47.26,"60,750.00",I
3,NER,7.50,0,"9,000","27,000",7.45,2022-05-17,2022-05-23,4,6.65,6.60,-1,6.65,8.65,9.59,"67,500.00",I
6,STA,29.50,0,"2,400","24,000",33.00,2022-05-20,2022-05-23,1,23.80,23.60,-2,23.80,50.50,27.79,"70,800.00",S
4,WHART,12.00,0,"10,000","30,000",11.70,2022-05-17,2022-05-23,4,10.70,10.50,-2,10.60,14.08,14.00,"120,000.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['DOHOME', 20.3, 0, 2400, 7200, 20.25, '2022-05-18', '2022-05-23',
        3, 19.6, 19.0, -6, 19.3, 25.62, 23.29, 48720.0, 'I'],
       ['JASIF', 12.0, 0, 10000, 110000, 10.1, '2022-05-17',
        '2022-05-23', 4, 11.1, 10.8, -3, 10.9, 11.3, 11.87, 120000.0,
        'I'],
       ['MAKRO', 40.5, 0, 1500, 7500, 40.4, '2022-05-19', '2022-05-23',
        2, 35.25, 36.75, 6, 36.75, 57.25, 47.26, 60750.0, 'I'],
       ['NER', 7.5, 0, 9000, 27000, 7.45, '2022-05-17', '2022-05-23', 4,
        6.65, 6.6, -1, 6.65, 8.65, 9.59, 67500.0, 'I'],
       ['WHART', 12.0, 0, 10000, 30000, 11.7, '2022-05-17', '2022-05-23',
        4, 10.7, 10.5, -2, 10.6, 14.08, 14.0, 120000.0, 'I'],
       ['IP', 20.3, 0, 2500, 7500, 20.2, '2022-05-20', '2022-05-23', 1,
        20.5, 20.4, -1, 20.7, 24.17, 25.25, 50750.0, 'I'],
       ['STA', 29.5, 0, 2400, 24000, 33.0, '2022-05-20', '2022-05-23', 1,
        23.8, 23.6, -2, 23.8, 50.5, 27.79, 70800.0, 'S']], dtype=object)

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([13, 11, 15, 18, 15, -1, 31])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,DOHOME,20.30,0,"2,400","7,200",20.25,2022-05-18,2022-05-23,3,19.60,19.00,-6,19.30,25.62,23.29,"48,720.00",I,13
1,JASIF,12.00,0,"10,000","110,000",10.10,2022-05-17,2022-05-23,4,11.10,10.80,-3,10.90,11.30,11.87,"120,000.00",I,11
2,MAKRO,40.50,0,"1,500","7,500",40.40,2022-05-19,2022-05-23,2,35.25,36.75,6,36.75,57.25,47.26,"60,750.00",I,15
3,NER,7.50,0,"9,000","27,000",7.45,2022-05-17,2022-05-23,4,6.65,6.60,-1,6.65,8.65,9.59,"67,500.00",I,18
4,WHART,12.00,0,"10,000","30,000",11.70,2022-05-17,2022-05-23,4,10.70,10.50,-2,10.60,14.08,14.00,"120,000.00",I,15
5,IP,20.30,0,"2,500","7,500",20.20,2022-05-20,2022-05-23,1,20.50,20.40,-1,20.70,24.17,25.25,"50,750.00",I,-1
6,STA,29.50,0,"2,400","24,000",33.00,2022-05-20,2022-05-23,1,23.80,23.60,-2,23.80,50.50,27.79,"70,800.00",S,31


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
11,ASP,"30,000",3.80,"114,000.00",0.40,"12,000.00",10.53,2
7,BCH,"18,000",21.30,"383,400.00",1.20,"21,600.00",5.63,4
19,DCC,"60,000",2.96,"177,600.00",0.19,"11,400.00",6.42,2
3,DIF,"50,000",14.70,"735,000.00",1.04,"52,150.00",7.10,2
9,DOHOME,"7,200",20.25,"145,800.00",0.00,0.00,0.00,4
14,HREIT,"40,000",8.95,"358,000.00",0.68,"27,080.00",7.56,2
23,IP,"7,500",20.20,"151,500.00",0.00,0.00,0.00,4
10,JASIF,"110,000",10.10,"1,111,000.00",0.94,"103,400.00",9.31,2
1,KCE,"14,000",72.50,"1,015,000.00",1.60,"22,400.00",2.21,3
5,MAKRO,"7,500",40.40,"303,000.00",0.72,"5,400.00",1.78,4


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,DOHOME,13,20.30,0,"2,400","7,200",20.25,19.60,19.00,-6,19.30,25.62,23.29,I,4,"48,720.00"
1,JASIF,11,12.00,0,"10,000","110,000",10.10,11.10,10.80,-3,10.90,11.30,11.87,I,2,"120,000.00"
2,MAKRO,15,40.50,0,"1,500","7,500",40.40,35.25,36.75,6,36.75,57.25,47.26,I,4,"60,750.00"
3,NER,18,7.50,0,"9,000","27,000",7.45,6.65,6.60,-1,6.65,8.65,9.59,I,4,"67,500.00"
4,WHART,15,12.00,0,"10,000","30,000",11.70,10.70,10.50,-2,10.60,14.08,14.00,I,2,"120,000.00"
5,IP,-1,20.30,0,"2,500","7,500",20.20,20.50,20.40,-1,20.70,24.17,25.25,I,4,"50,750.00"
6,STA,31,29.50,0,"2,400","24,000",33.00,23.80,23.60,-2,23.80,50.50,27.79,S,1,"70,800.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
IP,-1,20.3,0,2500,7500,20.20,20.50,20.40,-1,20.70,24.17,25.25,I,4,50750.0
JASIF,11,12.0,0,10000,110000,10.10,11.10,10.80,-3,10.90,11.30,11.87,I,2,120000.0
DOHOME,13,20.3,0,2400,7200,20.25,19.60,19.00,-6,19.30,25.62,23.29,I,4,48720.0
MAKRO,15,40.5,0,1500,7500,40.40,35.25,36.75,6,36.75,57.25,47.26,I,4,60750.0
WHART,15,12.0,0,10000,30000,11.70,10.70,10.50,-2,10.60,14.08,14.00,I,2,120000.0
NER,18,7.5,0,9000,27000,7.45,6.65,6.60,-1,6.65,8.65,9.59,I,4,67500.0
STA,31,29.5,0,2400,24000,33.00,23.80,23.60,-2,23.80,50.50,27.79,S,1,70800.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(one_file)